In [3]:
import requests
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# Load the data
loader = TextLoader('./10 Academy Cohort A - Weekly Challenge_ Week - 6.txt')
documents = loader.load()

# Chunk the data
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1764, which is longer than the specified 500
Created a chunk of size 1331, which is longer than the specified 500
Created a chunk of size 870, which is longer than the specified 500
Created a chunk of size 518, which is longer than the specified 500
Created a chunk of size 3478, which is longer than the specified 500
Created a chunk of size 856, which is longer than the specified 500
Created a chunk of size 1236, which is longer than the specified 500
Created a chunk of size 863, which is longer than the specified 500
Created a chunk of size 700, which is longer than the specified 500
Created a chunk of size 512, which is longer than the specified 500


In [9]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions
from dotenv import load_dotenv,find_dotenv

# Load OpenAI API key from .env file
load_dotenv(find_dotenv())

# Setup vector database
client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

# Populate vector database
vectorstore = Weaviate.from_documents(
    client = client,    
    documents = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False
)

# Define vectorstore as retriever to enable semantic search
retriever = vectorstore.as_retriever()

embedded weaviate is already listening on port 8079


/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/langchain_community/embeddings/openai.py:500: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)


In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# Define LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Define prompt template
template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use two sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
from datasets import Dataset

questions = ["What is the business objective of PromptlyTech?", 
             "What are the key services provided by PromptlyTech?",
             "What is prompt engineering?",
            ]
ground_truths = [["PromptlyTech aims to revolutionize how businesses interact with LLMs, making the technology more accessible, efficient, and effective."],
                ["PromptlyTech focuses on Automatic Prompt Generation, Automatic Evaluation Data Generation, and Prompt Testing and Ranking services."],
                ["Prompt engineering is the craft of designing queries or statements to guide LLMs to produce desired outcomes."]]
answers = []
contexts = []

# Inference
for query in questions:
  answers.append(rag_chain.invoke(query))
  contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# To dict
data = {
    "question": questions, 
    "answer": answers, 
    "contexts": contexts, 
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/langchain_community/embeddings/openai.py:500: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)
/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/langchain_community/chat_models/openai.py:458: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pyda

In [7]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

evaluating with [context_precision]


100%|██████████| 1/1 [00:06<00:00,  6.97s/it]
/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [context_recall]


100%|██████████| 1/1 [00:02<00:00,  2.04s/it]
/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [faithfulness]


100%|██████████| 1/1 [00:04<00:00,  4.81s/it]
/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [answer_relevancy]


  0%|          | 0/1 [00:00<?, ?it/s]/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/langchain_community/embeddings/openai.py:500: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)
100%|██████████| 1/1 [00:05<00:00,  5.58s/it]
/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemo

In [8]:
df

,question,answer,contexts,ground_truths,context_precision,context_recall,faithfulness,answer_relevancy
0,What is the business objective of PromptlyTech?,The business objective of PromptlyTech is to r...,[Business objective \nPromptlyTech is an inno...,[PromptlyTech aims to revolutionize how busine...,1.000000,1.0,1.0,1.0
1,What are the key services provided by Promptly...,The key services provided by PromptlyTech are ...,[Business objective \nPromptlyTech is an inno...,[PromptlyTech focuses on Automatic Prompt Gene...,0.750000,1.0,1.0,1.0
2,What is prompt engineering?,Prompt engineering is the craft of designing q...,[Prompt engineering is the craft of designing ...,[Prompt engineering is the craft of designing ...,0.805556,1.0,1.0,1.0
